<a href="https://colab.research.google.com/github/jai2shan/TSAI_END2/blob/main/Session%203/Session_3_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torchvision import datasets
datasets.MNIST('data', download=True)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train

In [2]:

# import torch
# train = torch.load("/content/data/MNIST/processed/training.pt")
# randInt = torch.randint(0,9,(60000,))

# import torch
# from torch.utils.data import DataLoader

# class CustomLoader_MNIST(DataLoader):
#     def __init__(self,pt_path):
#         self.path = pt_path
#         self.data = torch.load(self.path)
        

#     def __getitem__(self, index):
#       print(self.data)
#       r = self.data.iloc[index]
#       randInt = torch.randint(0,9,(r.shape[0],))

#       image, label = r
#       output = torch.cat((label.reshape(len(label),-1),
#                           (label + randInt).reshape(len(label),-1)),
#                          dim = 1)
#       x = torch.empty(size=(len(randInt), 28,28))
#       for i in range(len(randInt)):
#           x[i] = randInt[i]*torch.ones(28,28)
          
#       input = torch.cat((image, x),dim = 1)
#       return output, input

#     def __len__(self):
#       return len(self.data)

# train_data = CustomLoader_MNIST("/content/data/MNIST/processed/training.pt")
# test_data = CustomLoader_MNIST("/content/data/MNIST/processed/test.pt")

In [3]:
import torch
train = torch.load("/content/data/MNIST/processed/training.pt")
randInt = torch.randint(0,9,(60000,))

import torch
from torch.utils.data import DataLoader

class CustomLoader_MNIST(DataLoader):
    def __init__(self,pt_path):
        self.path = pt_path
        self.data = torch.load(self.path)
        

    def __getitem__(self, idx):
      if torch.is_tensor(idx):
        idx = idx.tolist()

      image, label = self.data[0][idx], self.data[1][idx] 

      randInt = torch.randint(0,9,(1,))
      x1, x2, y1, y2 = image.reshape(1,28,28), randInt, label, label + randInt

      return x1, x2, y1, y2

    def __len__(self):
      return len(self.data[0])

train_data = CustomLoader_MNIST("/content/data/MNIST/processed/training.pt")
test_data = CustomLoader_MNIST("/content/data/MNIST/processed/test.pt")

In [4]:
torch.manual_seed(1)
batch_size = 512
use_cuda = torch.cuda.is_available()

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, **kwargs)

In [5]:
from tqdm import tqdm
pbar = tqdm(train_loader)
device = torch.device("cuda" if use_cuda else "cpu")
for batch_idx, (x1, x2, y1, y2) in enumerate(pbar):
    x1, x2, y1, y2 = x1.to(device), x2.to(device), y1.to(device), y2.to(device)
    pbar.set_description(desc= f'batch_id={batch_idx}')

batch_id=117: 100%|██████████| 118/118 [00:03<00:00, 30.01it/s]


In [10]:
import torch.nn.functional as F
import torch.nn as nn
dropout_value = 0.1

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(dropout_value)
        ) # output_size = 26

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 24

        # TRANSITION BLOCK 1
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(1, 1), padding=0, bias=False),
        ) # output_size = 24
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12

        # CONVOLUTION BLOCK 2
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 10
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 8
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 6
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 6
        
        # OUTPUT BLOCK
        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=6)
        ) # output_size = 1

        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            # nn.BatchNorm2d(10),
            # nn.ReLU(),
            # nn.Dropout(dropout_value)
        ) 

        self.dropout = nn.Dropout(dropout_value)
        # self.relu = nn.ReLU()
        self.ln1 = nn.Linear(11,18)

    def forward(self, img, rnum):        
        img = self.convblock1(img)
        img = self.convblock2(img)
        img = self.convblock3(img)
        img = self.pool1(img)
        img = self.convblock4(img)
        img = self.convblock5(img)
        img = self.convblock6(img)
        img = self.convblock7(img)
        img = self.gap(img)        
        img = self.convblock8(img)
        
        img = img.view(-1, 10)
        rnum = torch.cat((img, rnum), dim=1)
        rnum = self.ln1(rnum)

        return F.log_softmax(img, dim=-1), F.log_softmax(rnum, dim=-1)

In [11]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
# summary(model, input_size=((1, 28, 28),1))

cuda


In [12]:
from tqdm import tqdm

train_losses = []
test_losses =  []
train_acc =  {'Image': [], 'RandomNumber': []}
test_acc = {'Image': [], 'RandomNumber': []}


def train(model, device, train_loader, optimizer, epoch):

  model.train()
  pbar = tqdm(train_loader)
  correct1 = 0
  processed = 0
  correct2 = 0

  for batch_idx, (x1, x2, y1, y2) in enumerate(pbar):
    # Data
    x1, x2, y1, y2 = x1.float().to(device), x2.float().to(device), y1.to(device), y2.to(device)
    # Init
    
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict

    y1_pred, y2_pred = model(x1, x2)

    # Calculate loss
    loss1 = F.nll_loss(y1_pred, y1)
    loss2 = F.nll_loss(y2_pred, y1)
    loss =  torch.add(loss1, loss2)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred1 = y1_pred.argmax(dim=1, keepdim=True)
    pred2 = y2_pred.argmax(dim=1, keepdim=True)
    
    # get the index of the max log-probability
    correct1 += pred1.eq(y1.view_as(pred1)).sum().item()
    correct2 += pred2.eq(y2.view_as(pred2)).sum().item()
    processed += len(x1)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy_digit={100*correct1/processed:0.2f} Accuracy_Rnum={100*correct2/processed:0.2f}')
    train_acc['Image'].append(100*correct1/processed)
    train_acc['RandomNumber'].append(100*correct2/processed)


In [13]:
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim

model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

EPOCHS = 20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)

  0%|          | 0/118 [00:00<?, ?it/s]

EPOCH: 0


Loss=4.400755882263184 Batch_id=117 Accuracy_digit=25.93 Accuracy_Rnum=3.00: 100%|██████████| 118/118 [00:02<00:00, 48.56it/s]
  0%|          | 0/118 [00:00<?, ?it/s]

EPOCH: 1


Loss=3.7185566425323486 Batch_id=117 Accuracy_digit=40.26 Accuracy_Rnum=5.46: 100%|██████████| 118/118 [00:02<00:00, 49.21it/s]
  0%|          | 0/118 [00:00<?, ?it/s]

...

Loss=0.30173081159591675 Batch_id=117 Accuracy_digit=97.05 Accuracy_Rnum=11.01: 100%|██████████| 118/118 [00:02<00:00, 47.90it/s]
  0%|          | 0/118 [00:00<?, ?it/s]

EPOCH: 18


Loss=0.22723326086997986 Batch_id=117 Accuracy_digit=97.19 Accuracy_Rnum=11.04: 100%|██████████| 118/118 [00:02<00:00, 48.71it/s]
  0%|          | 0/118 [00:00<?, ?it/s]

EPOCH: 19


Loss=0.2028593122959137 Batch_id=117 Accuracy_digit=97.30 Accuracy_Rnum=11.19: 100%|██████████| 118/118 [00:02<00:00, 47.45it/s]
